**Informaiton Retrieval Programming Assignment #2**
<br>**Real-world Indexing System**
<br>Build an inverted file that contains a postings list for each dictionary item.
<br>- Inverted file is written to disk as a binary file
<br>- Dictionary is also written to disk, which is in the form of a serialized object
<br>- For each word in the dictionary, a file offset to the corresponding on-disk posting list is stored (word as well)
<br>- Process the source text file only once
<br>- This program follows the memory-based inversion algorithm (Algorithm A)


<br><br>**Author:** Helen Ting He; **Date:** Sept 20, 2021

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
from collections import Counter, defaultdict, OrderedDict
import re
import time
import json

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# read original header file
header_url = 'http://pmcnamee.net/744/data/headlines.txt'
header = pd.read_csv(header_url,sep='\n',header=None)

In [ ]:
# Test files with 500 and 3000 paragraphs 
with open('Favorites.txt') as f:
    lines = f.readlines()
testfile = pd.DataFrame(list(zip(lines)))
with open('3000.txt') as f:
    lines2 = list(line for line in (l.strip() for l in f) if line)
testfile = pd.DataFrame(list(zip(lines)))
testfile2 = pd.DataFrame(list(zip(lines2)))

In [ ]:
def normalization(each_para):
  # To normalize each paragraph
  # @input: each phragraph, string
  # @output: counter of word for each paragraph, dict
  # normalized method:
  # 1. lower case 
  # 5. remove punctuation
  # 4. remove non-alpha letters (vocabulary freq:462,313)
  # 3. transform word numberals into numbers, then remove non-alpha again
  # 2. substitution of contractions
  # 6. remove single letter (vocabulary freq:157,136)
  # 7. remove any words contain * sign (vocabulary freq:156,954)
  # 8. remove any words contain + sign (vocabulary freq:156,776)
  # 9. remove any words starting or ending with - sign and ,sign (vocabulary freq:155,854)
  # 10. remove any words starting or ending with ..sign (vocabulary freq: 155,121)
  # 11. remove any words starting or ending with :sign, =sign, _sign,`sign,^^sign (vocabulary freq: 154,838)
  output_list_para = []
  counter_list_para = {}
  each_para = each_para.lower()
  each_para = each_para.replace("'re",' are').replace("'s",' is').replace("n't",' not')
  each_para = each_para.replace(' one',' 1').replace(' two',' 2').replace(' three',' 3').replace(' four',' 4').replace(' five ',' 5').replace(' six ',' 6').replace(' seven ',' 7').replace(' eight ',' 8').replace(' nine ',' 9 ').replace(' ten ',' 9 ')
  each_para = each_para.replace("'",'')
  for word in word_tokenize(each_para):
    if(re.search("\d",word)):
      continue
    if(re.search(r'\*', word)):
      continue
    if(re.search(r'\+', word)):
      continue
    if(re.search(r'^-', word) or re.search(r'-$', word) or re.search(r'^,', word) or re.search(r',$', word)) :
      continue
    if(re.search(r'^\..', word) or re.search(r'\..$', word)) :
      continue
    if(re.search(r'^:', word) or re.search(r':$', word) or re.search(r'^=', word) or re.search(r'=$', word) or re.search(r'^_', word) or re.search(r'_$', word) or re.search(r'^`', word) or re.search(r'`$', word) or re.search(r'^\^^', word) or re.search(r'\^^$', word)) :
      continue
    if(len(word)!= 1 ):
      output_list_para.append(word)   
      counter_list_para = Counter(output_list_para)
  return counter_list_para

In [ ]:
def readFile(whole_text): 
  ###############
  # MAIN METHOD1
  ###############
  # To read the text and process relevant methods
  # @input: input raw text, dataframe
  # @output: paragraph one by one, string
  i = 0
  input_line = []
  len_file = len(whole_text)
  output_wordlist_para = []
  output_wordlist_dict = {}
  while (1+3*i) < len_file:
    para = whole_text[0][1+3*i]
    output_wordlist_dict[i] = normalization(para)
    i = i + 1
  return output_wordlist_dict
  

In [ ]:
start = time.time()
normal_header = readFile(header)
print("--- %s seconds to buid normnalized counter ----"% (time.time() - start))

--- 187.51851320266724 seconds to buid normnalized counter ----


In [ ]:
# test print for docid = 0
normal_header[0]

Counter({'breakfast': 1,
         'club': 1,
         'for': 1,
         'gives': 1,
         'hunger': 1,
         'its': 1,
         'marching': 1,
         'orders': 1,
         'veterans': 1,
         'worcester': 1})

In [ ]:
def posting_list(docid_counter):
  # convert into posting list
  # @input:dict{ [docid]:dict{[word]: word freq per document} }, dict of dict
  # @ouput:dict{ [word]:dict{[docid]: term count} }, dict of dict
  posting_list_output = defaultdict(lambda: Counter([]))
  for doc, cnt in docid_counter.items():
    for word, word_cnt in cnt.items():
      posting_list_output[word][doc] += word_cnt
  return posting_list_output

In [ ]:
start = time.time()
posting_list_result = posting_list(normal_header)
print("--- %s seconds to buid posting list----"% (time.time() - start))

--- 7.137878179550171 seconds to buid posting list----


In [ ]:
posting_list_result['heidelberg']

Counter({114329: 1,
         135133: 1,
         174780: 1,
         221099: 1,
         243837: 1,
         452545: 1,
         491139: 1,
         491278: 1})

In [ ]:
# test return for 'daffodils' word
posting_list_result['^^']
posting_list_result['a-a']
posting_list_result['daffodils']

Counter({65342: 1, 267430: 1})

In [ ]:
def dictionary(posting_list):
  # To generate the unique sorted vocabulary list as key
  # DF and sorted offset as value 
  # @input: dict{ [word]:dict{[docid]: term count} }, dict of dict
  # @output: sorted unique vocabulary dict with DF and offset 
  sort_dict = {}
  result_sort_dict = {}
  offset_sum = 0
  offset_i = 0
  od = OrderedDict(sorted(posting_list.items()))
  sort_dict = OrderedDict(od)
  for i, word_i in enumerate(sort_dict.keys()):
    offset_i = len(sort_dict[word_i]) * 2 #offset
    result_sort_dict[word_i] = len(sort_dict[word_i].values()),offset_sum #DF
    offset_sum = offset_sum + offset_i 
  return result_sort_dict


In [ ]:
start = time.time()
dict_output = dictionary(posting_list_result)
print("--- %s seconds to buid dictionary----"% (time.time() - start))

--- 1.2549679279327393 seconds to buid dictionary----


In [ ]:
# save dictionary into disk
# convert it into json
with open('dictionary_header.txt', 'w') as f:
    f.write(json.dumps(dictionary(posting_list_result)))

In [ ]:
def inverted_file(key,posting_list):
  # To generate inverted file
  # @input: sorted key, list
  #         posting list, dict of dict
  # @output: inverted file, list
  inverted_list = []
  for word_i in key:
    for docid, cnt in posting_list[word_i].items():
      inverted_list.append(docid)
      inverted_list.append(cnt)
  return inverted_list


In [ ]:
# save inverted file in binary into disk
byte_file = inverted_file(dict_output.keys(),posting_list_result)
with open("inverted_fiile_binary.bin", "wb") as fb:
  for num in byte_file:
    fb.write(num.to_bytes(4, "big"))

In [ ]:
######## Testing ##########
# read dictionary file 
with open ("dictionary_header.txt","r+") as f:
  data = f.read()
dict_output = json.loads(data)

In [ ]:
# 1. Print out the document frequency and postings list for terms: “Heidelberg".
# Heidelberg: 
df = dict_output['Heidelberg'.lower()][0]
print("document frequency for Heidelberg: ", df)
posting_list_index = dict_output['Heidelberg'.lower()][1]
# get index of Heidelberg at dictionary key posiotion
index_heidelberg = list(dict_output.keys()).index('heidelberg')
next_word = list(dict_output)[index_heidelberg + 1]
# get index of posting list for next word
next_word_index = dict_output[next_word][1]
# index range of Heidelberg at inverted_file 
range_index = [posting_list_index, next_word_index]
print("index of range for Heidelberg: ", range_index)
# read inverted_file binary file every 4 byte
# then get their posting list 
list_num = []
with open("inverted_fiile_binary.bin", "br") as bf:
  for i in range(posting_list_index):
    data = bf.read(4)
  for i in range(posting_list_index, next_word_index):
    data = bf.read(4)
    number = int.from_bytes(data,"big")
    list_num.append(number)
print("posting list for Heidelberg: ")
print(list_num)

document frequency for Heidelberg:  8
index of range for Heidelberg:  [3203092, 3203108]
posting list for Heidelberg: 
[114329, 1, 135133, 1, 174780, 1, 221099, 1, 243837, 1, 452545, 1, 491139, 1, 491278, 1]


In [ ]:
#2. Give document frequency, but do not print postings for the words: "Hopkins", “Stanford", "Brown", and “college”
#(these postings lists are longer).
#Hokpins
df = dict_output['hopkins'][0]
print("document frequency for Hokpins: ", df)
#Stanford
df = dict_output['stanford'][0]
print("document frequency for Stanford: ", df)
#Brown
df = dict_output['brown'][0]
print("document frequency for brown: ", df)
#college
df = dict_output['college'][0]
print("document frequency for college: ", df)

document frequency for Hokpins:  71
document frequency for Stanford:  150
document frequency for brown:  770
document frequency for college:  1910


In [ ]:
#3. Print out the docids for documents that have both "Elon" and "Musk" in the text. 
#Elon
posting_list_index = dict_output['Elon'.lower()][1]
# get index of elon at dictionary key posiotion
index_elon = list(dict_output.keys()).index('elon')
next_word = list(dict_output)[index_elon + 1]
# get index of posting list for next word
next_word_index = dict_output[next_word][1]
# index range of elonat inverted_file 
range_index = [posting_list_index, next_word_index]
print("index of range for elon: ", range_index)
# read inverted_file binary file every 4 byte
# then get their posting list 
list_num = []
with open("inverted_fiile_binary.bin", "br") as bf:
  for i in range(posting_list_index):
    data = bf.read(4)
  for i in range(posting_list_index, next_word_index):
    data = bf.read(4)
    number = int.from_bytes(data,"big")
    list_num.append(number)
print("posting list for elon: ")
print(list_num)
#Musk
posting_list_index = dict_output['musk'.lower()][1]
# get index of musk at dictionary key posiotion
index_musk = list(dict_output.keys()).index('musk')
next_word = list(dict_output)[index_musk + 1]
# get index of posting list for next word
next_word_index = dict_output[next_word][1]
# index range of musk at inverted_file 
range_index = [posting_list_index, next_word_index]
print("index of range for musk: ", range_index)
# read inverted_file binary file every 4 byte
# then get their posting list 
list_num2 = []
with open("inverted_fiile_binary.bin", "br") as bf:
  for i in range(posting_list_index):
    data = bf.read(4)
  for i in range(posting_list_index, next_word_index):
    data = bf.read(4)
    number = int.from_bytes(data,"big")
    list_num2.append(number)
print("posting list for musk: ")
print(list_num2)
# find both elon and musk in the same file 
set_elon = set(list_num)
set_musk = set(list_num2)
print("find both elon and musk in the same file: ")
setone = {1}
set_same = set_elon.union(set_musk) - setone
print(set_same)

index of range for elon:  [2191074, 2191194]
posting list for elon: 
[3393, 1, 16330, 1, 19262, 1, 21341, 1, 29749, 1, 39287, 1, 44321, 1, 45978, 1, 52990, 1, 57023, 1, 57787, 1, 71988, 1, 84806, 1, 87959, 1, 98830, 1, 103398, 1, 104204, 1, 115207, 1, 122603, 1, 127050, 1, 128662, 1, 131441, 1, 131448, 1, 131514, 1, 135942, 1, 146965, 1, 151171, 1, 159147, 1, 186107, 1, 194998, 1, 197341, 1, 239304, 1, 240040, 1, 245923, 1, 249585, 1, 251252, 1, 274393, 1, 277539, 1, 283098, 1, 297139, 1, 301627, 1, 303775, 1, 305183, 1, 306988, 1, 307162, 1, 341755, 1, 342182, 1, 354346, 1, 369772, 1, 383528, 1, 399001, 1, 399946, 1, 420082, 1, 431495, 1, 431739, 1, 449684, 1, 456443, 1, 461816, 1, 479190, 1, 482769, 1]
index of range for musk:  [4788658, 4788764]
posting list for musk: 
[3393, 1, 16330, 1, 19262, 1, 21341, 1, 29749, 1, 44321, 1, 45978, 1, 52990, 1, 57023, 1, 57787, 1, 84806, 1, 98830, 1, 115207, 1, 122603, 1, 127050, 1, 128662, 1, 131448, 1, 131514, 1, 146965, 1, 159147, 1, 186107, 1